**Feature engineering for level 1 models**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

# **Librairies**

In [ ]:
import gc
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pandarallel import pandarallel

from matching import load_cleaned_data, distance
# from ressources import *
from fe import FE1
from dtypes import DTYPES_1, reduce_mem_usage
from fe_theo import feature_engineering_theo
from params import DEBUG, OUT_PATH, IS_TEST, RESSOURCES_PATH

pandarallel.initialize(nb_workers=12, progress_bar=False)
warnings.simplefilter("ignore")
pd.options.display.max_columns = 500

## Load Data

In [ ]:
if IS_TEST:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_test.csv")
    p1 = pd.read_csv(OUT_PATH + "p1_yv_test.csv")
    p2 = pd.read_csv(OUT_PATH + "p2_yv_test.csv")
else:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_train.csv")
    p1 = pd.read_csv(OUT_PATH + "p1_yv_train.csv")
    p2 = pd.read_csv(OUT_PATH + "p2_yv_train.csv")

In [ ]:
if DEBUG:
    p1 = p1.head(100000).copy()
    p2 = p2.head(100000).copy()

### Prepare

In [ ]:
train["idx"] = np.arange(len(train))

In [ ]:
# add other columns - needed for FE
cols = [
    "id",
    "name",
    "latitude",
    "longitude",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "categories",
    "category_simpl",
    "categories_split",
    "cat2",
    "idx",
    "state",
    "zip",
]
p1 = p1[["id"]].merge(train[cols], on="id", how="left")
p2 = p2[["id"]].merge(train[cols], on="id", how="left")

In [ ]:
# check for flipped sign on longitude - this may help test data a lot; test it? Move this code up to apply to "train"
dist = distance(
    np.array(p1["latitude"]),
    np.array(p1["longitude"]),
    np.array(p2["latitude"]),
    np.array(p2["longitude"]),
)
df = pd.DataFrame(dist)
df.columns = ["dist"]
df["dist"] = df["dist"].astype("int32")
df["dist1"] = (111173.444444444 * np.abs(p1["latitude"] - p2["latitude"])).astype(
    "int32"
)
df["dist2"] = np.sqrt(np.maximum(0, (1.0 * df["dist"]) ** 2 - df["dist1"] ** 2)).astype(
    "int32"
)
idx = (
    (df["dist1"] < 10000)
    & (df["dist2"] > 1000000)
    & (np.abs(p1["longitude"] + p2["longitude"]) < 0.1)
) & (p1["country"] == p2["country"])
# this selects only 3 cases in train data, but possibly more in test, so keep it becasue it is basically free
print("flipped sign of longitude for", idx.sum(), "points")
p1["longitude"].loc[idx] *= -1  # flip(correct) sign
del df, idx, dist
gc.collect()

### Youri & Vincent

In [ ]:
%%time
df = FE1(p1, p2)

In [ ]:
df.insert(0, "id_1", p1["id"].values)
df.insert(1, "id_2", p2["id"].values)

In [ ]:
df.head()

### Théo

In [ ]:
cols = [
    "id",
    "name",
    "latitude",
    "longitude",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "state",
    "zip",
    "categories",
    "idx",
]
pairs = pd.concat([p1[cols], p2[cols]], axis=1)
pairs.columns = [c + "_1" for c in cols] + [c + "_2" for c in cols]

In [ ]:
%%time

df_theo, fts_theo = feature_engineering_theo(pairs)

## Merge & Save

In [ ]:
df_merged = df.merge(df_theo, on=["id_1", "id_2"])

In [ ]:
df_merged = reduce_mem_usage(df_merged)

In [ ]:
if DEBUG:
    corrs = df_merged.corr()
    for col in corrs.columns:
        close = pd.DataFrame(corrs.loc[corrs[col] > 0.99][col])
        if len(close) > 1:
            display(close[close[col] < 1])

In [ ]:
if not DEBUG:
    if IS_TEST:
        df_merged.to_csv(OUT_PATH + "features_test_1.csv", index=False)
    else:
        df_merged.to_csv(OUT_PATH + "features_train_1.csv", index=False)